# 5.5. 读写文件

# 5.5.1. 加载和保存张量

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [4]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [5]:
# 存储张量列表并读取
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [6]:
# 存储张量字典并读取
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# 5.5.2. 加载和保存模型参数

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn((2, 20))
Y = net(X)
print(net)

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [8]:
# 保存模型参数
torch.save(net.state_dict(), 'mlp.params')

In [9]:
# 创建类对象并加载参数
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [10]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 5.5.3. 小结

save和load函数可用于张量对象的文件读写。

我们可以通过参数字典保存和加载网络的全部参数。

保存架构必须在代码中完成，而不是在参数中完成。

# 5.5.4. 练习

假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

torch.load('mlp.params')加载的是一个mlp参数，只要改为'hidden.params'就好

In [12]:
class MLP1(nn.Module):
    def __init__(self):
        super().__init__()
        self.output = nn.Linear(20, 256)
        
    def forward(self, X):
        return self.output(X)

clone_part = MLP1()
mlp_dict = torch.load('mlp.params')

temp = {'weight':net.state_dict()['hidden.weight'],'bias':net.state_dict()['hidden.bias']}
torch.save(temp,'hidden.params')
mlp_hidden_dict = torch.load('hidden.params')
print("传入参数字典为：")
print(mlp_dict)
print("传入第一层参数为：")
print(mlp_hidden_dict)

clone_part.load_state_dict(torch.load('hidden.params'),strict=False)
clone_part.eval()

传入参数字典为：
OrderedDict([('hidden.weight', tensor([[ 0.1864,  0.0333,  0.1993,  ..., -0.0204, -0.0569,  0.1486],
        [-0.2091, -0.0233, -0.0200,  ...,  0.2099, -0.1445, -0.0497],
        [-0.1450, -0.0436, -0.1642,  ..., -0.0716,  0.2026, -0.0758],
        ...,
        [ 0.1809,  0.0228,  0.1010,  ..., -0.1003,  0.1658,  0.0707],
        [-0.1288,  0.1635,  0.0067,  ..., -0.2007, -0.0681,  0.1416],
        [ 0.1655, -0.0204, -0.1610,  ...,  0.0170,  0.0091, -0.0609]])), ('hidden.bias', tensor([ 0.0237, -0.0862,  0.0789,  0.0789, -0.1755,  0.2051,  0.1058,  0.0100,
         0.1728, -0.0918, -0.0369, -0.0316, -0.0409, -0.1581,  0.1483,  0.0857,
         0.0377, -0.1134, -0.1761,  0.0600,  0.1407, -0.0715, -0.0274, -0.1015,
        -0.1523, -0.0132,  0.1089, -0.1958,  0.1569, -0.0553,  0.0255, -0.1272,
         0.1278,  0.1761,  0.2218, -0.1923, -0.0600,  0.1455, -0.1990,  0.1279,
        -0.0128,  0.0869,  0.1181,  0.1380,  0.0523, -0.1921, -0.2061, -0.1560,
        -0.1760, -0.2194,  0

MLP1(
  (output): Linear(in_features=20, out_features=256, bias=True)
)